In [52]:
import pyodbc
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.utils import to_categorical

In [2]:
class SQLConnection:
    """ SQL Connection Class

        Object that handles connection and querying of a database
    """

    def __init__(self, host, database):
        self.conn = pyodbc.connect(driver='{SQL Server}', server=host, database=database, trusted_connection='yes')

    def query_to_cursor(self, query_string):
        cursor = self.conn.cursor()
        cursor.execute(query_string)
        return cursor
    
    def query_to_df(self, query_string, index):
        df = pd.read_sql(query_string, conn.conn, index_col=index)
        return df

    def end(self):
        self.conn.close()

In [61]:
QUERY = 'SELECT * FROM gestures WHERE record BETWEEN %d AND %d;'

conn = SQLConnection(host='localhost', database='EMG_Gestures')

QUERY_SIZE = 'SELECT COUNT(record) FROM gestures;'

# dataframe = conn.query_to_df(QUERY_SIZE, '')

cursor_size = conn.query_to_cursor(QUERY_SIZE)
data_size = cursor_size.fetchval()

batch_number = 256

batch_size = data_size / batch_number

# a = 0
# b = 4096

# data = (a,b)

#data = conn.query_to_df(QUERY % data, 'record')


In [50]:
    a = 0
    b = 10
    
    data_range = (a,b)
    
    data = conn.query_to_df(QUERY % data_range, 'record')
    
    data = data.abs()
    
    # Check head rows
    print(data.head())
    
    # drop class from data for training
    train_X = data.drop(columns=['class'])
    
    #train_X = MinMaxScaler()
    rows = len(train_X)
    cols = len(train_X.columns)
    
    pre_scl = train_X.values.reshape(rows*cols, 1)
    
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(pre_scl)
    train_X_res = scaler.transform(train_X)
    
    train_X = train_X_res.reshape(rows, cols)
    
    train_X = pd.DataFrame(train_X)
    
    # Check head rows
    train_X.head()
    
    # convert to binarise 7 classes from 'class' column
    train_y = to_categorical(data['class'], num_classes = 8)
    
    # Check head rows after conversion
    print(train_y[0:5])
    
    print(train_X)

        channel1  channel2  channel3  channel4  channel5  channel6  channel7  \
record                                                                         
1        0.00083   0.00004   0.00031   0.00095   0.00061   0.00026   0.00013   
2        0.00001   0.00003   0.00003   0.00019   0.00007   0.00013   0.00002   
3        0.00001   0.00001   0.00001   0.00006   0.00001   0.00000   0.00001   
4        0.00002   0.00003   0.00004   0.00001   0.00005   0.00005   0.00005   
5        0.00011   0.00002   0.00004   0.00005   0.00013   0.00004   0.00002   

        channel8  class  
record                   
1        0.00002    2.0  
2        0.00000    5.0  
3        0.00000    1.0  
4        0.00005    4.0  
5        0.00010    3.0  
[[0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]]
          0         1         2         3         4         5         6  \
0  0.747368 -0.915789 -0.347368  1.000000  0.2

In [ ]:
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import EarlyStopping



#batch_size = 4096
#batch_number = 256
num_classes = 8
epochs = 20



model = Sequential()

#get number of columns in training data
#n_cols = train_X.shape[1]

model.add(Dense(10, activation='relu', input_shape=(8,)))
model.add(Dropout(0.2))
model.add(Dense(10, activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(num_classes, activation='softmax'))

#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=3)

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

model.summary()

for i in range(round(batch_number*0.8)):
    print("Batch: ")
    print(i)
    a = i*batch_size 
    b = (i+1)*batch_size 
    
    data_range = (a,b)
    
    data = conn.query_to_df(QUERY % data_range, 'record')
    
    data = data.abs()
    
    # Check head rows
    #print(data.head())
    
    # drop class from data for training
    train_X = data.drop(columns=['class'])
    
    #train_X = MinMaxScaler()
    rows = len(train_X)
    cols = len(train_X.columns)
    
    pre_scl = train_X.values.reshape(rows*cols, 1)
    
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(pre_scl)
    train_X_res = scaler.transform(train_X)
    
    train_X = train_X_res.reshape(rows, cols)
    
    train_X = pd.DataFrame(train_X)
    
    # Check head rows
    train_X.head()
    
    # convert to binarise 7 classes from 'class' column
    train_y = to_categorical(data['class'], num_classes = 8)
    
    # Check head rows after conversion
    train_y[0:5]

    
    baseline_history = model.fit(train_X, train_y, epochs=30, validation_split=0.3, callbacks=[early_stopping_monitor])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_64 (Dense)             (None, 10)                90        
_________________________________________________________________
dropout_47 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_65 (Dense)             (None, 10)                110       
_________________________________________________________________
dropout_48 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_66 (Dense)             (None, 10)                110       
_________________________________________________________________
dropout_49 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_67 (Dense)             (None, 8)                 88        
Total para

2867/2867 [==============================]2867/2867 [==============================] - 1s 283us/step - loss: 1.5844 - acc: 0.3408 - val_loss: 1.4951 - val_acc: 0.4439

Epoch 10/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 271us/step - loss: 1.5651 - acc: 0.3568 - val_loss: 1.4932 - val_acc: 0.4504

Epoch 11/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 298us/step - loss: 1.5726 - acc: 0.3425 - val_loss: 1.4922 - val_acc: 0.4463

Epoch 12/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 262us/step - loss: 1.5628 - acc: 0.3404 - val_loss: 1.4764 - val_acc: 0.4439

Epoch 13/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 276us/step - loss: 1.5725 - acc: 0.3540 - val_loss: 1.4739 - val_acc: 0.4488

Epoch 14/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 277us/step

2867/2867 [==============================]2867/2867 [==============================] - 1s 283us/step - loss: 1.4604 - acc: 0.4081 - val_loss: 1.3433 - val_acc: 0.5033

Epoch 25/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 300us/step - loss: 1.4489 - acc: 0.4070 - val_loss: 1.3315 - val_acc: 0.5138

Epoch 26/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 331us/step - loss: 1.4417 - acc: 0.4217 - val_loss: 1.3347 - val_acc: 0.5081

Epoch 27/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 279us/step - loss: 1.4466 - acc: 0.4130 - val_loss: 1.3338 - val_acc: 0.5081

Epoch 28/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 238us/step - loss: 1.4426 - acc: 0.4070 - val_loss: 1.3246 - val_acc: 0.5203

Epoch 29/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 286us/step

2867/2867 [==============================]2867/2867 [==============================] - 1s 264us/step - loss: 1.3884 - acc: 0.4430 - val_loss: 1.2731 - val_acc: 0.5317

Epoch 2/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 206us/step - loss: 1.3808 - acc: 0.4489 - val_loss: 1.2701 - val_acc: 0.5325

Epoch 3/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 248us/step - loss: 1.3749 - acc: 0.4580 - val_loss: 1.2696 - val_acc: 0.5382

Epoch 4/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 321us/step - loss: 1.3804 - acc: 0.4587 - val_loss: 1.2623 - val_acc: 0.5317

Epoch 5/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 287us/step - loss: 1.3698 - acc: 0.4527 - val_loss: 1.2622 - val_acc: 0.5325

Epoch 6/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 278us/step - lo

2867/2867 [==============================]2867/2867 [==============================] - 1s 309us/step - loss: 1.3906 - acc: 0.4395 - val_loss: 1.2403 - val_acc: 0.5480

Epoch 2/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 328us/step - loss: 1.3922 - acc: 0.4405 - val_loss: 1.2416 - val_acc: 0.5537

Epoch 3/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 303us/step - loss: 1.4080 - acc: 0.4322 - val_loss: 1.2444 - val_acc: 0.5447

Epoch 4/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 345us/step - loss: 1.3983 - acc: 0.4409 - val_loss: 1.2415 - val_acc: 0.5593

Batch: 
13
Train on 2867 samples, validate on 1230 samples
Epoch 1/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 286us/step - loss: 1.3473 - acc: 0.4806 - val_loss: 1.2376 - val_acc: 0.5577

Epoch 2/30
2867/2867 [==============================]2867

...

2867/2867 [==============================]2867/2867 [==============================] - 1s 328us/step - loss: 1.2941 - acc: 0.4977 - val_loss: 1.1641 - val_acc: 0.5626

Epoch 3/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 283us/step - loss: 1.2949 - acc: 0.4848 - val_loss: 1.1608 - val_acc: 0.5724

Epoch 4/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 265us/step - loss: 1.2996 - acc: 0.4925 - val_loss: 1.1605 - val_acc: 0.5675

Epoch 5/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 293us/step - loss: 1.3026 - acc: 0.4737 - val_loss: 1.1655 - val_acc: 0.5602

Epoch 6/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 244us/step - loss: 1.2941 - acc: 0.4845 - val_loss: 1.1659 - val_acc: 0.5626

Epoch 7/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 300us/step - lo

2867/2867 [==============================]2867/2867 [==============================] - 1s 314us/step - loss: 1.3102 - acc: 0.4806 - val_loss: 1.2038 - val_acc: 0.5309

Epoch 6/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 269us/step - loss: 1.3225 - acc: 0.4806 - val_loss: 1.1986 - val_acc: 0.5415

Epoch 7/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 318us/step - loss: 1.2970 - acc: 0.4883 - val_loss: 1.2074 - val_acc: 0.5333

Batch: 
42
Train on 2867 samples, validate on 1230 samples
Epoch 1/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 335us/step - loss: 1.3024 - acc: 0.4932 - val_loss: 1.1498 - val_acc: 0.5512

Epoch 2/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 272us/step - loss: 1.2933 - acc: 0.4813 - val_loss: 1.1572 - val_acc: 0.5480

Epoch 3/30
2867/2867 [==============================]2867

2867/2867 [==============================]2867/2867 [==============================] - 1s 314us/step - loss: 1.3029 - acc: 0.4887 - val_loss: 1.1626 - val_acc: 0.5577

Epoch 8/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 268us/step - loss: 1.2984 - acc: 0.4789 - val_loss: 1.1620 - val_acc: 0.5610

Epoch 9/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 265us/step - loss: 1.2912 - acc: 0.4880 - val_loss: 1.1642 - val_acc: 0.5553

Epoch 10/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 300us/step - loss: 1.2862 - acc: 0.4901 - val_loss: 1.1613 - val_acc: 0.5537

Epoch 11/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 303us/step - loss: 1.3045 - acc: 0.4789 - val_loss: 1.1640 - val_acc: 0.5528

Epoch 12/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 289us/step -

2867/2867 [==============================]2867/2867 [==============================] - 1s 236us/step - loss: 1.2862 - acc: 0.4998 - val_loss: 1.1432 - val_acc: 0.5585

Epoch 6/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 297us/step - loss: 1.2782 - acc: 0.4974 - val_loss: 1.1420 - val_acc: 0.5659

Batch: 
53
Train on 2867 samples, validate on 1230 samples
Epoch 1/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 316us/step - loss: 1.2964 - acc: 0.4981 - val_loss: 1.1612 - val_acc: 0.5447

Epoch 2/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 290us/step - loss: 1.2783 - acc: 0.4970 - val_loss: 1.1699 - val_acc: 0.5415

Epoch 3/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 262us/step - loss: 1.2789 - acc: 0.4887 - val_loss: 1.1568 - val_acc: 0.5545

Epoch 4/30
2867/2867 [==============================]2867

2867/2867 [==============================]2867/2867 [==============================] - 1s 314us/step - loss: 1.2902 - acc: 0.4890 - val_loss: 1.1422 - val_acc: 0.5707

Batch: 
59
Train on 2867 samples, validate on 1230 samples
Epoch 1/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 257us/step - loss: 1.2972 - acc: 0.4970 - val_loss: 1.1382 - val_acc: 0.5480

Epoch 2/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 292us/step - loss: 1.3097 - acc: 0.4908 - val_loss: 1.1587 - val_acc: 0.5520

Epoch 3/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 322us/step - loss: 1.2725 - acc: 0.5023 - val_loss: 1.1350 - val_acc: 0.5537

Epoch 4/30
2867/2867 [==============================]2867/2867 [==============================] - 1s 328us/step - loss: 1.2779 - acc: 0.4887 - val_loss: 1.1401 - val_acc: 0.5504

Epoch 5/30
2867/2867 [==============================]2867

In [74]:
def plot_history(histories, key='binary_crossentropy'):
  plt.figure(figsize=(16,10))
    
  for name, history in histories:
    val = plt.plot(history.epoch, history.history['val_'+key],
                   '--', label=name.title()+' Val')
    plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
             label=name.title()+' Train')

  plt.xlabel('Epochs')
  plt.ylabel(key.replace('_',' ').title())
  plt.legend()

  plt.xlim([0,max(history.epoch)])


plot_history([('baseline', baseline_history)])
              #,
              #('smaller', smaller_history),
              #('bigger', bigger_history)])

NameError: name 'baseline_history' is not defined

In [ ]:
conn.close()

In [20]:
df = pd.DataFrame(np.arange(12).reshape(3,4), columns=['A', 'B', 'C', 'D'])
print(df)
df = df.drop(columns=['B', 'C'])
print(df)

   A  B   C   D
0  0  1   2   3
1  4  5   6   7
2  8  9  10  11
   A   D
0  0   3
1  4   7
2  8  11
